In [25]:
import pandas as pd
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [26]:
min_len = 900
max_len = 16000
min_ltr_len = 85
max_ltr_len = 5000

In [27]:
df = pd.read_csv("data/results/all.gff", sep="\t", header=None, comment="#")
df.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']
print(len(df.index))

250784


In [28]:
df.sort_values(['start', 'end'], inplace=True)

In [29]:
df = df[df.attribute.str.contains('LTR_id')]
print(len(df.index))

188553


In [30]:
df = df[(df.end - df.start) <= max_len]
print(len(df.index))

188314


In [31]:
df = df[(df.end - df.start) >= min_len]
print(len(df.index))

129378


In [32]:
df.head()

,seqname,source,feature,start,end,score,strand,frame,attribute
7774,chr00,ST_annotation,TE,429,2347,.,+,.,LTR_id:4994;source_name:RLC_family136_LTR_7650...
0,ChrUn,ST_annotation,TE,18405,21310,.,+,.,LTR_id:4487;source_name:RLG_singleton_family95...
7775,chr00,ST_annotation,TE,20847,25597,.,+,.,LTR_id:3831;source_name:RLG_LTR_63_FAM000516;t...
52097,chr02,ST_annotation,TE,25059,27683,.,+,.,LTR_id:5770;source_name:RLG_LTR_259_FAM002147;...
121659,chr06,ST_annotation,TE,30914,31867,.,+,.,LTR_id:6092;source_name:RLG_LTR_428_FAM004707;...


In [35]:
records = {}
fasta_seq = SeqIO.parse('data/fasta/potato_dm_v404_all_pm_un.fasta', 'fasta')
for record in fasta_seq:
    records[record.id] = ''.join(str(record.seq).splitlines())


In [ ]:
for k,v in df.iterrows():
    #cut the genome in order to extract the LTR sequence
    record_seq_clipped = records[v.seqname][v.start:v.end]
    half_seq_len = (v.end - v.start) / 2
    record_seq_clipped_1 = record_seq_clipped[: half_seq_len - 1]
    record_seq_clipped_2 = record_seq_clipped[half_seq_len:]
    seq_1 = SeqRecord(Seq(record_seq_clipped_1), id="1", description="yeah!")
    seq_2 = SeqRecord(Seq(record_seq_clipped_2), id="2", description="yeah!")
    SeqIO.write(seq_1, "q.fasta", "fasta")
    SeqIO.write(seq_2, "s.fasta", "fasta")
    cols = ['qseqid','sseqid','qstart','qend','sstart','send','score','length','mismatch','gaps','gapopen','nident','pident','evalue','qlen','slen','qcovs']
    outfmt = "'6 %s'" % (' '.join(cols), )
    cline = NcbiblastnCommandline(query="q.fasta", subject="s.fasta", task="blastn", strand="plus",outfmt=outfmt)
    stdout, stderr = cline()
    stdout = stdout.strip()
    if stdout != "":
        vals = stdout.split('\t')
        result = dict(zip(cols, vals))
        length, gaps, mismatch = int(result['length']), int(result['gaps']), int(result['mismatch'])
        if length >= min_len and length <= max_len:
            print(v.attribute, length)
            
        else:
            continue
#            print('no bitch', result['length'])

